**Install python dependencies**

In [ ]:
!pip install -q -r ./requirements.txt

**Load python libraries**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

%matplotlib inline
plt.style.use('seaborn')

**Data Mining**

In [ ]:
# df_events = pd.read_csv('events_prep.csv')
# df_odds = pd.read_csv('odds_prep.csv')

# df = pd.merge(df_events, df_odds)

# df.to_csv('data.csv', index=False)

df = pd.read_csv('data.csv')

df

**Drop Betting Odds for Baseline Model**

In [ ]:
df = df[['name', 'matchday', 'event_type', 'count']]

df_players = df.groupby("name")

for player_tuple in df_players:
    player_name = player_tuple[0]
    df_player = player_tuple[1]

    df_player = df_player.sort_values(by=['matchday', 'event_type'])
    
    # df_events = df_player.groupby("event_type")

    # for event_tuple in df_events:
    #     event_name = event_tuple[0]
    #     df_event = event_tuple[1]

    #     print(df_event)

    #     for i

In [ ]:
df = pd.read_csv('data_rl9.csv')

df_events = df.groupby("event_type")

for event_tuple in df_events:
    event_name = event_tuple[0]
    df_event = event_tuple[1]

    if event_name == 'pass':
        df_event.to_csv('data_rl9_pass.csv')

    # 34 matchdays
    for i in range(1,30):
        label = i + 5;

        # label + 1 because range is exclusive (]
        values = df_event.loc[df_event['matchday'].isin(range(i,label+1))]['count'].values.tolist()
        


        

        # matchdays = list(range(i, label))

        # for matchday in matchdays:





In [ ]:
df = pd.read_csv('data_rl9_pass.csv')

df['occurences'] = df['count']

df = df[['name', 'matchday', 'event_type', 'occurences']]

df.set_index('matchday', inplace=True)

df.plot(color='green', linewidth=3, figsize=(12,6))

In [ ]:
# simple moving average of 5 and 10
df['SMA_5'] = df.occurences.rolling(5, min_periods=1).mean()
df['SMA_10'] = df.occurences.rolling(10, min_periods=1).mean()

colors = ['green', 'red', 'purple']

df[['occurences', 'SMA_5', 'SMA_10']].plot(color=colors, linewidth=3, figsize=(12,6))
# --> SMA_5


In [ ]:
# cumulative moving average
df['CMA'] = df.occurences.expanding().mean()

colors = ['green', 'orange']

df[['occurences', 'CMA']].plot(color=colors, linewidth=3, figsize=(12,6))

In [ ]:
# exponential moving average with different alphas
df['EMA_0.1'] = df.occurences.ewm(alpha=0.1, adjust=False).mean()
df['EMA_0.3'] = df.occurences.ewm(alpha=0.3, adjust=False).mean()
df['EMA_0.5'] = df.occurences.ewm(alpha=0.5, adjust=False).mean()

# --> the value itself
df['EMA_1'] = df.occurences.ewm(alpha=1, adjust=False).mean()

colors = ['green', 'blue', 'orchid', 'pink', 'black']

df[['occurences', 'EMA_0.1', 'EMA_0.3', 'EMA_0.5', 'EMA_1']].plot(color=colors, linewidth=3, figsize=(12,6), alpha=0.8)
# --> EMA_0.5

In [ ]:
# compare averages
colors = ['green', 'red', 'orange', 'orchid']

df[['occurences', 'SMA_5', 'CMA', 'EMA_0.5']].plot(color=colors, linewidth=3, figsize=(12,6), alpha=0.8)

# --> EMA_0.5


In [ ]:
columns = ['SMA_5', 'SMA_10', 'CMA', 'EMA_0.1', 'EMA_0.3', 'EMA_0.5', 'occurences']

df_reg_acc = pd.DataFrame(columns= ['Regression', 'MAE', 'MSE', 'RMSE', 'R-Squared'])

for column in columns:
    pred_column_name = column + '_pred'
    df[pred_column_name] = df[column].shift(1)

    # drop first 10 rows, learning
    df_pred = df.tail(df.index.max() - 10)

    occurences = df_pred['occurences'].to_numpy()
    predictions = df_pred[pred_column_name].to_numpy()

    mae = metrics.mean_absolute_error(occurences, predictions)
    mse = metrics.mean_squared_error(occurences, predictions)
    rmse = np.sqrt(mse) 
    r2 = metrics.r2_score(occurences, predictions)

    df_reg_acc = df_reg_acc.append({'Regression': column, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R-Squared': r2}, ignore_index=True)

df_reg_acc

In [ ]:
def calculate_averages(df):
    df['occurences'] = df['count']
    df.set_index('matchday', inplace=True)

    # simple moving average of 5 and 10
    df['SMA_5'] = df.occurences.rolling(5, min_periods=1).mean()
    df['SMA_10'] = df.occurences.rolling(10, min_periods=1).mean()

    # cumulative moving average
    df['CMA'] = df.occurences.expanding().mean()

    # exponential moving average with different alphas
    df['EMA_0.1'] = df.occurences.ewm(alpha=0.1, adjust=False).mean()
    df['EMA_0.3'] = df.occurences.ewm(alpha=0.3, adjust=False).mean()
    df['EMA_0.5'] = df.occurences.ewm(alpha=0.5, adjust=False).mean()

    return df

def calculate_regression_accuracies(df, df_reg_acc, columns):

    player_name = df['name'].max()
    event_type = df['event_type'].max()

    for column in columns:
        pred_column_name = column + '_pred'
        df[pred_column_name] = df[column].shift(1)

        # drop first 10 rows, learning
        df_pred = df.tail(df.index.max() - 10)

        occurences = df_pred['occurences'].to_numpy()
        predictions = df_pred[pred_column_name].to_numpy()

        mae = metrics.mean_absolute_error(occurences, predictions)
        mse = metrics.mean_squared_error(occurences, predictions)
        rmse = np.sqrt(mse) 
        r2 = metrics.r2_score(occurences, predictions)

        df_reg_acc = df_reg_acc.append({'player_name': player_name, 'event_type': event_type, 'Regression': column, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R-Squared': r2}, ignore_index=True)

    return df_reg_acc

In [99]:
df_reg_acc = pd.DataFrame(columns= ['player_name', 'event_type', 'Regression', 'MAE', 'MSE', 'RMSE', 'R-Squared'])
columns = ['SMA_5', 'SMA_10', 'CMA', 'EMA_0.1', 'EMA_0.3', 'EMA_0.5', 'occurences']

df = pd.read_csv('data.csv')

# drop betting odds
df = df[['name', 'matchday', 'event_type', 'count']]

df_players = df.groupby("name")

for player_tuple in df_players:
    player_name = player_tuple[0]
    df_player = player_tuple[1]

    df_player = df_player.sort_values(by=['matchday', 'event_type'])

    df_events = df_player.groupby("event_type")
    
    for event_tuple in df_events:
        event_name = event_tuple[0]
        df_event = event_tuple[1]
        if df_event.shape[0] < 34:

            # !!! SOMEHOW PASS,29 FOR EXAMPLE IS MISSING IN PREPROCESSING, LEADING TO ERRORS!!!
            print(df_event.shape[0])
            print(event_name)
            print(player_name)
        # df_event = calculate_averages(df_event)
        # df_reg_acc = calculate_regression_accuracies(df_event, df_reg_acc, columns)

df_reg_acc.to_csv('reg_acc.csv')

33
awardedPenalty
 Aarón
33
blockedGoalShot
 Aarón
33
causedPenalty
 Aarón
33
cross
 Aarón
33
defended
 Aarón
33
doublePack
 Aarón
33
error
 Aarón
33
farCorner
 Aarón
33
foul
 Aarón
33
freeKick
 Aarón
33
goal
 Aarón
33
goalAgainst
 Aarón
33
goalAssist
 Aarón
33
goalMissedFar
 Aarón
33
hattrick
 Aarón
33
interception
 Aarón
33
lostPenalty
 Aarón
33
missedChance
 Aarón
33
offside
 Aarón
33
ownGoal
 Aarón
33
pass
 Aarón
33
penaltyGoal
 Aarón
2
playerOff
 Aarón
2
playerOn
 Aarón
33
redCard
 Aarón
33
savedPenalty
 Aarón
33
secondYellowCard
 Aarón
33
shotAtGoal
 Aarón
33
successfulTackle
 Aarón
33
superPass
 Aarón
33
throwIn
 Aarón
33
unsuccessfulPass
 Aarón
33
unsuccessfulPenalty
 Aarón
33
unsuccessfulTackle
 Aarón
33
yellowCard
 Aarón
8
playerOff
 Iago
2
playerOff
 Tuta
3
playerOn
 Tuta
2
playerOff
 Wendell
7
playerOn
 Wendell
33
awardedPenalty
 William
33
causedPenalty
 William
33
defended
 William
33
doublePack
 William
33
goal
 William
33
goalAgainst
 William
33
goalAssist
 William
33
h